In [ ]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

In [ ]:
# Data Sources

confirmedCases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
deathCases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
recoveryCases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'


In [ ]:
# Import data

confirmedRaw_df = pd.read_csv(confirmedCases_url)
deathsRaw_df = pd.read_csv(deathCases_url)
recoveriesRaw_df = pd.read_csv(recoveryCases_url)

N=confirmedRaw_df.index.shape[0]

In [ ]:
# Create a dataframe for one country

# Calculate
country_str = 'Greece'

cases_bool_ser=confirmedRaw_df['Country/Region']==country_str
confirmed_df = confirmedRaw_df[cases_bool_ser].transpose()[4:]

deaths_bool_ser=deathsRaw_df['Country/Region']==country_str
deaths_df = deathsRaw_df[deaths_bool_ser].transpose()[4:]

recoveries_bool_ser=recoveriesRaw_df['Country/Region']==country_str
recoveries_df = recoveriesRaw_df[recoveries_bool_ser].transpose()[4:]

totalInfected_np = confirmed_df.to_numpy() - recoveries_df.to_numpy() - deaths_df.to_numpy()
totalInfected_df = pd.DataFrame(totalInfected_np.flatten(),index=confirmed_df.index)

cases_df = pd.concat(
    [
        confirmed_df, 
        deaths_df, 
        recoveries_df,
        totalInfected_df,
    ],
    axis=1
)

cases_df.columns=['Confirmed','Deaths','Recoveries','Infected']
cases_df.tail()

In [ ]:
# Presentation Parameters

xticks_np = np.arange(5, cases_df.shape[0]-1, step=30)
xticks_np = np.append(xticks_np,cases_df.shape[0]-1)
xlabels_list = cases_df.index[xticks_np]
color_list=['b','r','g','m']

In [ ]:
# Present

ax = cases_df.plot(
    figsize=(30,10),
    logy=True,
    color=color_list,
)
ax.legend(cases_df.columns)

# Weekly Gridlines
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(7))
ax.grid(axis='both', which='minor')

# 30-day ticks
plt.xticks(xticks_np, xlabels_list)

# Labels
plt.ylabel('Log Case Count', fontsize=20)
plt.xlabel('Time (date)', fontsize=20)
plt.title('COVID Timeline in '+country_str, fontsize=20)


In [ ]:
# Daily new cases

# Calculate
diffCases_df = cases_df.diff()


In [ ]:
# Present

# Ommit infected from the plot for ease
ax = diffCases_df[['Confirmed','Deaths','Recoveries']].plot(
    figsize=(30,10),
    logy=True,
    color=color_list,
)
ax.legend(cases_df.columns)

# Weekly Gridlines
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(7))
ax.grid(axis='both', which='minor')

# 30-day ticks
plt.xticks(xticks_np, xlabels_list)

# Labels
plt.ylabel('Log Case Count', fontsize=20)
plt.xlabel('Time (date)', fontsize=20)
plt.title('COVID Timeline in '+country_str, fontsize=20)

In [ ]:
# Rolling average

# Calculate
casesRolling_df = cases_df.rolling(7).mean()


In [ ]:
# Present

ax = casesRolling_df.plot(
    figsize=(30,10),
    logy=True,
    color=color_list,
)
ax.legend(cases_df.columns)

# Weekly Gridlines
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(7))
ax.grid(axis='both', which='minor')

# 30-day ticks
plt.xticks(xticks_np, xlabels_list)

# Labels
plt.ylabel('Log Case Count', fontsize=20)
plt.xlabel('Time (date)', fontsize=20)
plt.title('7day Average Cases for '+country_str, fontsize=20)

In [ ]:
# Case Fatality Rate Over time

# Calculate

fatality_ser = casesRolling_df.Deaths / casesRolling_df.Confirmed * 100


In [ ]:
# Present

ax = fatality_ser.plot(
    figsize=(30,10),
)

# Weekly Gridlines
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(7))
ax.yaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1))
ax.grid(axis='both', which='minor')

# 30-day ticks
plt.xticks(xticks_np, xlabels_list)

# Labels
plt.ylabel('Fatality rate (%)', fontsize=20)
plt.xlabel('Time (date)', fontsize=20)
plt.title('Fatality rate for '+country_str, fontsize=20)

In [ ]:
colors_np=np.arange(0,casesRolling_df.shape[0])

plt.figure(figsize=(30,10))
ax = plt.scatter(
    casesRolling_df.Infected,
    fatality_ser,
    c=colors_np,
    cmap=plt.cm.get_cmap('Blues'),
)
#plt.gca().set_xscale('log')
plt.grid()
cbar = plt.colorbar(ax, )
cbar.ax.set_yticklabels(xlabels_list)  # vertically oriented colorbar

plt.ylabel('Fatality %', fontsize=20)
plt.xlabel('Confirmed Cases', fontsize=20)
plt.title('Fatality Rate versus Confirmed Cases for '+country_str, fontsize=20)

In [ ]:
# Compare Countries

# Calculate
country_str_list = [
    'Greece', 
#    'Russia',
#    'US',
#    'China',
    'Sweden',
    'Italy', 
#    'Turkey',
#    'Albania',
    'Germany',
]

# hard-code the respective country population
countryPop_list = [
    10473455,
#    145872256,
#    329064917,
#    1433783686,
    10036379,
    60550075,
#    83429615,
#    2880917,
    83122889,
]

# Compare cases per population
confirmedComparison_df = confirmedRaw_df.groupby(['Country/Region']).sum().drop(['Lat', 'Long'], axis=1).transpose()[country_str_list]
casesPerPop_df = confirmedComparison_df / countryPop_list

# Compare deaths per population
deathsComparison_df = deathsRaw_df.groupby(['Country/Region']).sum().drop(['Lat', 'Long'], axis=1).transpose()[country_str_list]
deathsPerPop_df = deathsComparison_df / countryPop_list *100


In [ ]:
# Present

ax = confirmedComparison_df.plot(
    figsize=(30,10),
    logy=True,
)

# Weekly Gridlines
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(7))
ax.grid(axis='both', which='minor')

# 30-day ticks
plt.xticks(xticks_np, xlabels_list)

# Labels
plt.ylabel('Log Case Count', fontsize=20)
plt.xlabel('Time (date)', fontsize=20)
plt.title('Confirmed cases comparison by Country', fontsize=20)

In [ ]:
# Present

ax = casesPerPop_df.plot(
    figsize=(30,10),
   logy=True,
)

# Weekly Gridlines
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(7))
ax.grid(axis='both', which='minor')

# 30-day ticks
plt.xticks(xticks_np, xlabels_list)

# Labels
plt.ylabel('Log Case Rate', fontsize=20)
plt.xlabel('Time (date)', fontsize=20)
plt.title('Confirmed cases per Population', fontsize=20)

In [ ]:
# Present

ax = deathsPerPop_df.plot(
    figsize=(30,10),
#    logy=True,
)

# Weekly Gridlines
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(7))
ax.grid(axis='both', which='minor')

# 30-day ticks
plt.xticks(xticks_np, xlabels_list)

# Labels
plt.ylabel('Mortality %', fontsize=20)
plt.xlabel('Time (date)', fontsize=20)
plt.title('Mortality Rate comparison', fontsize=20)